In [2]:
import bnlearn
import numpy as np

# Sample data points from a DAG

In [3]:
# import bayesian graph with conditional probability distribution
DAG = bnlearn.import_DAG('./data/child.bif', CPD=True)
df = bnlearn.sampling(DAG, n=10000)
df.head()

[bnlearn] >Loading bif file <./data/child.bif>


Generating for node: Disease:   0%|          | 0/20 [00:00<?, ?it/s]      

[bnlearn] >Checking CPDs..
[bnlearn] >Warning: CPD [Age] does not add up to 1 but is: [[1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[bnlearn] >Warning: CPD [ChestXray] does not add up to 1 but is: [[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
[bnlearn] >Warning: CPD [HypoxiaInO2] does not add up to 1 but is: [[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
[bnlearn] >Warning: CPD [LowerBodyO2] does not add up to 1 but is: [[1. 1. 1.]
 [1. 1. 1.]]
[bnlearn] >Warning: CPD [RUQO2] does not add up to 1 but is: [1. 1. 1.]
[bnlearn] >Warning: CPD [XrayReport] does not add up to 1 but is: [1. 1. 1. 1. 1.]
[bnlearn] >Check for DAG structure. Correct: True
[bnlearn] >Forward sampling for 10000 samples..


Generating for node: LVHreport: 100%|██████████| 20/20 [00:07<00:00,  2.70it/s]    


,BirthAsphyxia,Disease,Sick,Age,LungFlow,LungParench,Grunting,GruntingReport,ChestXray,XrayReport,CO2,CO2Report,CardiacMixing,HypoxiaInO2,RUQO2,DuctFlow,HypDistrib,LowerBodyO2,LVH,LVHreport
0,1,3,1,0,0,0,1,1,0,4,0,0,2,1,2,0,0,2,0,0
1,1,3,1,0,1,0,1,1,2,0,0,0,2,1,1,0,0,1,0,1
2,1,1,1,1,2,0,1,1,0,0,0,0,3,2,0,1,0,0,1,1
3,1,2,1,1,0,0,1,1,0,2,0,0,2,1,1,0,0,1,0,0
4,1,2,1,2,1,0,1,1,1,1,0,0,2,1,1,1,0,1,1,1


[bnlearn] >Plot based on BayesianModel


{'pos': {'DuctFlow': array([-0.01157793, -1.        ]),
  'HypDistrib': array([ 0.53872215, -0.77039267]),
  'CardiacMixing': array([-0.33733289,  0.98362427]),
  'HypoxiaInO2': array([-0.79783728, -0.56224487]),
  'LungParench': array([-0.76428758,  0.54626644]),
  'CO2': array([0.81589777, 0.57936283]),
  'ChestXray': array([ 0.85947617, -0.00256029]),
  'LungFlow': array([-0.57196537, -0.80454144]),
  'Grunting': array([ 0.79501513, -0.6249765 ]),
  'Sick': array([-0.27766832, -0.86777722]),
  'LVH': array([-0.60013575,  0.79665091]),
  'LVHreport': array([0.56959814, 0.82552189]),
  'LowerBodyO2': array([ 0.94591546, -0.30450867]),
  'RUQO2': array([-0.01873429,  0.9753773 ]),
  'CO2Report': array([-0.97029144,  0.31903946]),
  'XrayReport': array([-0.86690414, -0.27245835]),
  'BirthAsphyxia': array([0.97795258, 0.24114213]),
  'Disease': array([ 0.31851433, -0.97629343]),
  'GruntingReport': array([0.27656368, 0.8899295 ]),
  'Age': array([-0.88092041,  0.02883871])},
 'G': <netw

In [10]:
bnlearn.plot(DAG)

In [11]:
# check column values
for column in df.columns:
    print(column, set(df[column].values))

BirthAsphyxia {0, 1}
Disease {0, 1, 2, 3, 4, 5}
Sick {0, 1}
Age {0, 1, 2}
LungFlow {0, 1, 2}
LungParench {0, 1, 2}
Grunting {0, 1}
GruntingReport {0, 1}
ChestXray {0, 1, 2, 3, 4}
XrayReport {0, 1, 2, 3, 4}
CO2 {0, 1, 2}
CO2Report {0, 1}
CardiacMixing {0, 1, 2, 3}
HypoxiaInO2 {0, 1, 2}
RUQO2 {0, 1, 2}
DuctFlow {0, 1, 2}
HypDistrib {0, 1}
LowerBodyO2 {0, 1, 2}
LVH {0, 1}
LVHreport {0, 1}


In [4]:
# target label comes from GruntingReport node
y = df["GruntingReport"].values

In [7]:
# add domain label at 1 index in y
y = np.stack((y, df["Disease"].values), 1)

In [8]:
# remaining nodes form the input features
x_df = df.drop(["GruntingReport", "Disease"],axis=1) 

In [15]:
cols = list(x_df.columns)

In [26]:
# save the x and y values in npz
np.savez('./data/child5Domains.npz', x=x_df.values, y=y)

# Create the dag_mat file

In [12]:
dag_mat = np.zeros((len(x_df.columns),len(x_df.columns)))

In [20]:
dag_mat[cols.index("BirthAsphyxia")][cols.index("Age")]=1
dag_mat[cols.index("BirthAsphyxia")][cols.index("Sick")]=1
dag_mat[cols.index("BirthAsphyxia")][cols.index("DuctFlow")]=1
dag_mat[cols.index("BirthAsphyxia")][cols.index("CardiacMixing")]=1
dag_mat[cols.index("BirthAsphyxia")][cols.index("LungParench")]=1
dag_mat[cols.index("BirthAsphyxia")][cols.index("LungFlow")]=1
dag_mat[cols.index("BirthAsphyxia")][cols.index("LVH")]=1
dag_mat[cols.index("Sick")][cols.index("Age")]=1
dag_mat[cols.index("Sick")][cols.index("Grunting")]=1
dag_mat[cols.index("DuctFlow")][cols.index("HypDistrib")]=1
dag_mat[cols.index("CardiacMixing")][cols.index("HypDistrib")]=1
dag_mat[cols.index("CardiacMixing")][cols.index("HypoxiaInO2")]=1
dag_mat[cols.index("LungParench")][cols.index("HypoxiaInO2")]=1
dag_mat[cols.index("LungParench")][cols.index("Grunting")]=1
dag_mat[cols.index("LungParench")][cols.index("CO2")]=1
dag_mat[cols.index("LungParench")][cols.index("ChestXray")]=1
dag_mat[cols.index("LungFlow")][cols.index("ChestXray")]=1
dag_mat[cols.index("LVH")][cols.index("LVHreport")]=1
dag_mat[cols.index("ChestXray")][cols.index("XrayReport")]=1
dag_mat[cols.index("CO2")][cols.index("CO2Report")]=1
dag_mat[cols.index("HypoxiaInO2")][cols.index("RUQO2")]=1
dag_mat[cols.index("HypoxiaInO2")][cols.index("LowerBodyO2")]=1
dag_mat[cols.index("HypDistrib")][cols.index("LowerBodyO2")]=1

In [21]:
np.savez('./data/dag_mat_child5Domains.npz', mat=dag_mat)

# 3 Domains Data

In [37]:
# Creating dataset for 3 source domains
df = bnlearn.sampling(DAG, n=10000)

Generating for node: Disease:   0%|          | 0/20 [00:00<?, ?it/s]      

[bnlearn] >Forward sampling for 10000 samples..


Generating for node: LVHreport: 100%|██████████| 20/20 [00:07<00:00,  2.64it/s]    


In [29]:
df

,BirthAsphyxia,Disease,Sick,Age,LungFlow,LungParench,Grunting,GruntingReport,ChestXray,XrayReport,CO2,CO2Report,CardiacMixing,HypoxiaInO2,RUQO2,DuctFlow,HypDistrib,LowerBodyO2,LVH,LVHreport
0,1,5,0,0,2,1,0,0,3,3,2,1,1,1,2,0,0,2,1,1
1,1,3,1,1,1,2,0,0,4,4,2,1,2,2,0,0,0,1,0,0
2,1,3,1,0,1,0,1,1,0,0,0,0,2,1,0,0,0,1,0,0
3,1,3,1,0,0,0,1,1,0,0,0,0,2,1,0,0,0,1,0,0
4,0,2,1,2,1,0,1,1,1,1,1,0,1,1,1,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1,2,1,1,1,0,1,1,1,1,0,0,2,1,0,0,0,0,1,1
9996,1,2,0,0,1,0,1,1,1,3,0,0,2,2,1,0,0,0,1,1
9997,1,1,1,0,2,0,1,1,2,0,0,0,3,2,1,1,0,1,1,1
9998,1,3,1,0,1,0,1,0,1,1,1,0,2,1,1,0,0,1,0,0


In [38]:
y = df["GruntingReport"].values

In [39]:
# add domain labels
y = np.stack((y, df["CardiacMixing"].values),1)

In [40]:
x_df = df.drop(["GruntingReport", "CardiacMixing"],axis=1) 

In [41]:
cols = list(x_df.columns)

In [34]:
np.savez('./data/child3Domains.npz', x=x_df.values, y=y)

# Create dag mat file

In [42]:
dag_mat[cols.index("BirthAsphyxia")][cols.index("Disease")]=1
dag_mat[cols.index("Disease")][cols.index("Age")]=1
dag_mat[cols.index("Disease")][cols.index("Sick")]=1
dag_mat[cols.index("Disease")][cols.index("DuctFlow")]=1
dag_mat[cols.index("Disease")][cols.index("LungParench")]=1
dag_mat[cols.index("Disease")][cols.index("LungFlow")]=1
dag_mat[cols.index("Disease")][cols.index("LVH")]=1
dag_mat[cols.index("Sick")][cols.index("Age")]=1
dag_mat[cols.index("Sick")][cols.index("Grunting")]=1
dag_mat[cols.index("DuctFlow")][cols.index("HypDistrib")]=1
dag_mat[cols.index("Disease")][cols.index("HypDistrib")]=1
dag_mat[cols.index("Disease")][cols.index("HypoxiaInO2")]=1
dag_mat[cols.index("LungParench")][cols.index("HypoxiaInO2")]=1
dag_mat[cols.index("LungParench")][cols.index("Grunting")]=1
dag_mat[cols.index("LungParench")][cols.index("CO2")]=1
dag_mat[cols.index("LungParench")][cols.index("ChestXray")]=1
dag_mat[cols.index("LungFlow")][cols.index("ChestXray")]=1
dag_mat[cols.index("LVH")][cols.index("LVHreport")]=1
dag_mat[cols.index("ChestXray")][cols.index("XrayReport")]=1
dag_mat[cols.index("CO2")][cols.index("CO2Report")]=1
dag_mat[cols.index("HypoxiaInO2")][cols.index("RUQO2")]=1
dag_mat[cols.index("HypoxiaInO2")][cols.index("LowerBodyO2")]=1
dag_mat[cols.index("HypDistrib")][cols.index("LowerBodyO2")]=1

In [43]:
np.savez('./data/dag_mat_child3Domains.npz', mat=dag_mat)

# Create a DAG manually

In [5]:
# Define the network structure
edges = [('Cloudy', 'Sprinkler'),
         ('Cloudy', 'Rain'),
         ('Sprinkler', 'Wet_Grass'),
         ('Rain', 'Wet_Grass')]

# Make the actual Bayesian DAG
DAG = bnlearn.make_DAG(edges)

[bnlearn] >Bayesian DAG created.


[bnlearn] >Plot based on BayesianModel


{'pos': {'Cloudy': array([ 0.54531726, -1.        ]),
  'Sprinkler': array([0.85643477, 0.57072186]),
  'Rain': array([-0.75980028, -0.47498882]),
  'Wet_Grass': array([-0.64195175,  0.90426696])},
 'G': <networkx.classes.digraph.DiGraph at 0x1347d86d8>}

In [6]:
bnlearn.plot(DAG)

# Adding Conditional Probability Distribtution (CPD) to the causal structure

In [9]:
from pgmpy.factors.discrete import TabularCPD

# Cloudy
cpt_cloudy = TabularCPD(variable='Cloudy', variable_card=2, values=[[0.5], [0.5]])
print(cpt_cloudy)

# Sprinkler
cpt_sprinkler = TabularCPD(variable='Sprinkler', variable_card=2,
                           values=[[0.5, 0.9],
                                   [0.5, 0.1]],
                           evidence=['Cloudy'], evidence_card=[2])
print(cpt_sprinkler)

# Rain
cpt_rain = TabularCPD(variable='Rain', variable_card=2,
                      values=[[0.8, 0.2],
                              [0.2, 0.8]],
                      evidence=['Cloudy'], evidence_card=[2])
print(cpt_rain)

# Wet Grass
cpt_wet_grass = TabularCPD(variable='Wet_Grass', variable_card=2,
                           values=[[1, 0.1, 0.1, 0.01],
                                   [0, 0.9, 0.9, 0.99]],
                           evidence=['Sprinkler', 'Rain'],
                           evidence_card=[2, 2])
print(cpt_wet_grass)

+-----------+-----+
| Cloudy(0) | 0.5 |
+-----------+-----+
| Cloudy(1) | 0.5 |
+-----------+-----+
+--------------+-----------+-----------+
| Cloudy       | Cloudy(0) | Cloudy(1) |
+--------------+-----------+-----------+
| Sprinkler(0) | 0.5       | 0.9       |
+--------------+-----------+-----------+
| Sprinkler(1) | 0.5       | 0.1       |
+--------------+-----------+-----------+
+---------+-----------+-----------+
| Cloudy  | Cloudy(0) | Cloudy(1) |
+---------+-----------+-----------+
| Rain(0) | 0.8       | 0.2       |
+---------+-----------+-----------+
| Rain(1) | 0.2       | 0.8       |
+---------+-----------+-----------+
+--------------+--------------+--------------+--------------+--------------+
| Sprinkler    | Sprinkler(0) | Sprinkler(0) | Sprinkler(1) | Sprinkler(1) |
+--------------+--------------+--------------+--------------+--------------+
| Rain         | Rain(0)      | Rain(1)      | Rain(0)      | Rain(1)      |
+--------------+--------------+--------------+-------

In [10]:
# connect DAG to CPD
DAG = bnlearn.make_DAG(DAG, CPD=[cpt_cloudy, cpt_sprinkler, cpt_rain, cpt_wet_grass])

[bnlearn] >No changes made to existing Bayesian DAG.
[bnlearn] >Add CPD: Cloudy
[bnlearn] >Add CPD: Sprinkler
[bnlearn] >Add CPD: Rain
[bnlearn] >Add CPD: Wet_Grass
[bnlearn] >Checking CPDs..
[bnlearn] >Check for DAG structure. Correct: True


[bnlearn] >Plot based on BayesianModel


{'pos': {'Cloudy': array([0.60907946, 1.        ]),
  'Sprinkler': array([ 0.89010065, -0.56255354]),
  'Rain': array([-0.69088491, -0.92655431]),
  'Wet_Grass': array([-0.8082952 ,  0.48910786])},
 'G': <networkx.classes.digraph.DiGraph at 0x1344c05c0>}

In [12]:
bnlearn.plot(DAG)

# Inference on DAG

In [11]:
q1 = bnlearn.inference.fit(DAG, variables=['Wet_Grass'], evidence={'Rain':1, 'Sprinkler':0, 'Cloudy':1})

Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]

[bnlearn] >Variable Elimination..
+--------------+------------------+
| Wet_Grass    |   phi(Wet_Grass) |
+==============+==================+
| Wet_Grass(0) |           0.1000 |
+--------------+------------------+
| Wet_Grass(1) |           0.9000 |
+--------------+------------------+


# Sample data from asia graph

In [3]:
# import bayesian graph with conditional probability distribution
DAG = bnlearn.import_DAG('asia', CPD=True)
df = bnlearn.sampling(DAG, n=2000)
df.head()

[bnlearn] >Loading bif file <//anaconda3/lib/python3.7/site-packages/bnlearn/data/ASIA/asia.bif>


Generating for node: lung:  25%|██▌       | 2/8 [00:00<00:00, 15.28it/s] 

[bnlearn] >Checking CPDs..
[bnlearn] >Check for DAG structure. Correct: True
[bnlearn] >Forward sampling for 2000 samples..


Generating for node: xray: 100%|██████████| 8/8 [00:00<00:00,  8.66it/s]  


,smoke,bronc,lung,asia,tub,either,dysp,xray
0,0,0,1,1,1,1,0,1
1,0,0,1,1,1,1,0,1
2,0,1,1,1,1,1,1,1
3,0,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1


In [4]:
# check column values
for column in df.columns:
    print(column, set(df[column].values))

smoke {0, 1}
bronc {0, 1}
lung {0, 1}
asia {0, 1}
tub {0, 1}
either {0, 1}
dysp {0, 1}
xray {0, 1}


In [5]:
# target label comes from GruntingReport node
y = df["xray"].values

In [6]:
# add domain label at 1 index in y
labels = []

for i,j in zip(df["asia"].values, df["smoke"].values):
    if i==0 and j==0:
        labels.append(0)
    elif i==0 and j==1:
        labels.append(1)
    elif i==1 and j==0:
        labels.append(2)
    else:
        labels.append(3)
        
y = np.stack((y, labels), 1)

In [7]:
# remaining nodes form the input features
x_df = df.drop(["asia", "smoke", "xray"],axis=1) 

In [8]:
cols = list(x_df.columns)

In [9]:
cols

['bronc', 'lung', 'tub', 'either', 'dysp']

In [11]:
# save the x and y values in npz
np.savez('./data/asia4Domains.npz', x=x_df.values, y=y)

In [12]:
dag_mat = np.zeros((len(x_df.columns),len(x_df.columns)))

In [13]:
dag_mat[cols.index("bronc")][cols.index("dysp")]=1
dag_mat[cols.index("lung")][cols.index("either")]=1
dag_mat[cols.index("tub")][cols.index("either")]=1
dag_mat[cols.index("either")][cols.index("dysp")]=1

In [14]:
dag_mat.shape

(5, 5)

In [15]:
np.savez('./data/dag_mat_asia4Domains.npz', mat=dag_mat)